<a href="https://colab.research.google.com/github/Prithvi-Prabhu/breastcancerVITmodel/blob/main/VIT16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install tensorflow opencv-python matplotlib

# Import libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Change the path based on your dataset location
dataset_path = '/content/drive/MyDrive/KAU-BCMD'


Mounted at /content/drive


In [ ]:
def apply_clahe(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    image = clahe.apply(image)
    return image

def preprocess_images(image_dir, target_size=(224, 224)):
    images, labels = [], []
    class_labels = os.listdir(image_dir)
    for label in class_labels:
        label_path = os.path.join(image_dir, label)
        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)
            image = cv2.imread(img_path)
            if image is not None:
                image = apply_clahe(image)
                image = cv2.resize(image, target_size)
                images.append(image)
                labels.append(class_labels.index(label))
    return np.array(images), np.array(labels)


In [ ]:
X, y = preprocess_images(dataset_path)
X = X / 255.0  # Normalize the images
y = tf.keras.utils.to_categorical(y, num_classes=4)  # One-hot encode labels

# Shuffle and split the dataset
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape to include channel dimension for grayscale images
X_train = X_train.reshape(-1, 224, 224, 1)
X_val = X_val.reshape(-1, 224, 224, 1)


In [ ]:
from tensorflow.keras.layers import Dense, LayerNormalization, MultiHeadAttention, Dropout, Input, Embedding, Reshape, Conv2D, GlobalAveragePooling1D
from tensorflow.keras.models import Model
import tensorflow as tf

def vision_transformer(input_shape=(224, 224, 1), num_classes=4, patch_size=16, num_heads=4, transformer_units=128):
    # Input layer
    inputs = Input(shape=input_shape)

    # Create patches
    patches = Conv2D(
        filters=transformer_units,
        kernel_size=patch_size,
        strides=patch_size,
        padding="valid"
    )(inputs)

    # Flatten patches
    num_patches = (input_shape[0] // patch_size) * (input_shape[1] // patch_size)  # Static computation
    patches = Reshape((num_patches, transformer_units))(patches)

    # Positional encoding
    positions = tf.range(start=0, limit=num_patches, delta=1)  # Static range
    position_embeddings = Embedding(input_dim=num_patches, output_dim=transformer_units)(positions)
    patches = patches + position_embeddings  # Add positional encoding

    # Transformer blocks
    for _ in range(4):  # 4 Transformer blocks
        # Multi-head self-attention
        attention_output = MultiHeadAttention(
            num_heads=num_heads,
            key_dim=transformer_units,
            dropout=0.1
        )(patches, patches)
        # Add & Normalize
        attention_output = LayerNormalization(epsilon=1e-6)(patches + attention_output)
        # Feed Forward Network (FFN)
        ffn_output = Dense(transformer_units, activation="relu")(attention_output)
        ffn_output = Dense(transformer_units)(ffn_output)
        # Add & Normalize
        patches = LayerNormalization(epsilon=1e-6)(attention_output + ffn_output)

    # Classification head
    representation = GlobalAveragePooling1D()(patches)
    representation = Dropout(0.2)(representation)
    outputs = Dense(num_classes, activation="softmax")(representation)

    # Model
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Create the model
model = vision_transformer(input_shape=(224, 224, 1), num_classes=4, patch_size=16)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 14, 14, 128)    │         32,896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 196, 128)       │              0 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 196, 128)       │              0 │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 196, 128)       │        263,808 │ add[0][0], add[0][0]   │
│ (MultiHeadAttention)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 196, 128)       │              0 │ add[0][0],             │
│                           │                        │                │ multi_head_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 196, 128)       │            256 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 196, 128)       │         16,512 │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 196, 128)       │         16,512 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 196, 128)       │              0 │ layer_normalization[0… │
│                           │                        │                │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 196, 128)       │            256 │ add_2[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 196, 128)       │        263,808 │ layer_normalization_1… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_3 (Add)               │ (None, 196, 128)       │              0 │ layer_normalization_1… │
│                           │                        │                │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 196, 128)       │            256 │ add_3[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 196, 128)       │         16,512 │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)      

 Total params: 1,222,788 (4.66 MB)

 Trainable params: 1,222,788 (4.66 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=25,
    batch_size=32
)


Epoch 1/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 38s 350ms/step - accuracy: 0.9310 - loss: 0.4139 - val_accuracy: 0.9373 - val_loss: 0.2703
Epoch 2/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - accuracy: 0.9321 - loss: 0.3148 - val_accuracy: 0.9373 - val_loss: 0.2698
Epoch 3/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.9412 - loss: 0.2846 - val_accuracy: 0.9373 - val_loss: 0.2745
Epoch 4/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.9448 - loss: 0.2584 - val_accuracy: 0.9373 - val_loss: 0.2736
Epoch 5/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.9449 - loss: 0.2582 - val_accuracy: 0.9373 - val_loss: 0.2681
Epoch 6/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.9433 - loss: 0.2526 - val_accuracy: 0.9373 - val_loss: 0.2799
Epoch 7/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.9336 - loss: 0.2864 - val_accuracy: 0.9373 - val_loss: 0.2617
Epoch 8/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.9490 - loss: 0.2267 - val_accuracy: 0.9373 -

In [ ]:
results = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Loss: {results[0]}, Validation Accuracy: {results[1]}")


Validation Loss: 0.2728615403175354, Validation Accuracy: 0.9373433589935303


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20%
    height_shift_range=0.2,  # Randomly shift images vertically by 20%
    zoom_range=0.2,  # Randomly zoom in/out on images
    horizontal_flip=True  # Randomly flip images horizontally
)

# Fit the generator on your training data
datagen.fit(X_train)

# Use it during model training
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=25
)


Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


50/50 ━━━━━━━━━━━━━━━━━━━━ 32s 336ms/step - accuracy: 0.9456 - loss: 0.2426 - val_accuracy: 0.9373 - val_loss: 0.2689
Epoch 2/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 22s 151ms/step - accuracy: 0.9408 - loss: 0.2625 - val_accuracy: 0.9373 - val_loss: 0.2714
Epoch 3/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 9s 145ms/step - accuracy: 0.9344 - loss: 0.2776 - val_accuracy: 0.9373 - val_loss: 0.2711
Epoch 4/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 9s 121ms/step - accuracy: 0.9302 - loss: 0.2922 - val_accuracy: 0.9373 - val_loss: 0.2783
Epoch 5/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 9s 162ms/step - accuracy: 0.9429 - loss: 0.2548 - val_accuracy: 0.9373 - val_loss: 0.2845
Epoch 6/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 125ms/step - accuracy: 0.9377 - loss: 0.2780 - val_accuracy: 0.9373 - val_loss: 0.2713
Epoch 7/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 142ms/step - accuracy: 0.9406 - loss: 0.2564 - val_accuracy: 0.9373 - val_loss: 0.2700
Epoch 8/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 128ms/step - accuracy: 0.9329 - loss: 0.2781 - val_accuracy: 0.9373 - va

In [ ]:
results = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Loss: {results[0]}, Validation Accuracy: {results[1]}")

Validation Loss: 0.26792845129966736, Validation Accuracy: 0.9373433589935303
